### beta 버전 추가 파트:
analyze_face_pose(keypoints) 함수 추가

수평 변화 감지: 귀와 귀를 잇는 선의 각도
     
높이 변화 감지: 코와 목 사이의 거리

수직 변화 감지: 코와 목을 잇는 선의 각도
    
좌우 변화 감지: 코와 양쪽 귀의 중점을 잇는 선의 각도
    

# 0. Install and Import Dependencies

In [1]:
#!pip install tensorflow==2.16.1 opencv-python matplotlib

In [2]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import time

# 1. Load Model + Draw Keypoints + Draw Edges

In [3]:
interpreter = tf.lite.Interpreter(model_path='movenet_lighting_tflite_float16.tflite')
interpreter.allocate_tensors()

In [4]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

In [5]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [6]:
# 어깨 중간과 엉덩이 중간에 세로선 그리기 추가
def draw_midline(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    shoulder_mid = (shaped[5][:2] + shaped[6][:2]) / 2
    hip_mid = (shaped[11][:2] + shaped[12][:2]) / 2

    if (shaped[5][2] > confidence_threshold and shaped[6][2] > confidence_threshold and
            shaped[11][2] > confidence_threshold and shaped[12][2] > confidence_threshold):
        cv2.line(frame, (int(shoulder_mid[1]), int(shoulder_mid[0])), (int(hip_mid[1]), int(hip_mid[0])), (255, 0, 0), 2)


In [7]:
def draw_face_axis(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    # 얼굴의 중심축을 찾기 위한 키포인트 정의
    right_eye = shaped[2][:2]
    left_eye = shaped[1][:2]
    right_ear = shaped[4][:2]
    left_ear = shaped[3][:2]

    # 눈의 중간 지점(코로 가정)과 귀의 중간 지점 계산
    nose = (right_eye + left_eye) / 2
    ears_mid = (right_ear + left_ear) / 2

    # 눈의 중간 지점과 귀의 중간 지점을 연결하는 선 그리기
    if (shaped[2][2] > confidence_threshold and shaped[1][2] > confidence_threshold and
        shaped[4][2] > confidence_threshold and shaped[3][2] > confidence_threshold):
        cv2.line(frame, (int(nose[1]), int(nose[0])), (int(ears_mid[1]), int(ears_mid[0])), (0, 0, 255), 2)

In [8]:
def draw_face_vertical_line(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    # 머리 상단과 목 위치를 사용하여 대략적인 이마와 턱의 위치 추정
    head_top = shaped[0][:2]  # 머리 상단(이마로 가정)
    neck = shaped[1][:2]  # 목(턱 근처로 가정)

    if (shaped[0][2] > confidence_threshold and shaped[1][2] > confidence_threshold):
        # 이마에서 턱까지의 선 그리기
        cv2.line(frame, (int(head_top[1]), int(head_top[0])), (int(neck[1]), int(neck[0])), (0, 0, 255), 2)


In [9]:
def is_address_pose(keypoints_with_scores, confidence_threshold=0.4):
    keypoints = np.squeeze(keypoints_with_scores)
    
    # 키포인트 신뢰도 체크
    if (keypoints[5][2] < confidence_threshold or keypoints[6][2] < confidence_threshold or
        keypoints[11][2] < confidence_threshold or keypoints[12][2] < confidence_threshold):
        return False  # 신뢰도가 임계값 미만인 키포인트가 있으면 어드레스 자세로 판단하지 않음
    
    # 어깨와 골반의 중간점 계산
    shoulder_midpoint = (keypoints[5][:2] + keypoints[6][:2]) / 2
    hip_midpoint = (keypoints[11][:2] + keypoints[12][:2]) / 2

    # print("어드레스")
    
    # 어깨와 골반의 높이 차이 계산
    vertical_diff = abs(shoulder_midpoint[1] - hip_midpoint[1])
    
    # 높이 차이 임계값을 조정하여 수평성 체크
    vertical_diff_threshold = 5  # 높이 차이 임계값을 5로 조정 (상황에 따라 조정 가능)
    if vertical_diff < vertical_diff_threshold:
        return True  # 어깨와 골반의 높이 차이가 임계값 이하이면 어드레스 자세로 판별
    else:
        return False

In [10]:
def calculate_angle(p1, p2):
    """두 점 p1, p2 간의 각도를 계산합니다."""
    angle = np.arctan2(p2[1] - p1[1], p2[0] - p1[0]) * 180.0 / np.pi
    return angle

In [11]:
def is_body_straight(keypoints_with_scores, confidence_threshold=0.4):
    keypoints = np.squeeze(keypoints_with_scores)
    head_point = keypoints[0, :2]
    neck_point = keypoints[1, :2]
    hip_center = (keypoints[11, :2] + keypoints[12, :2]) / 2
    
    # print("바디바디")
    
    # 머리-목 각도와 목-골반 각도를 계산
    head_neck_angle = calculate_angle(head_point, neck_point)
    neck_hip_angle = calculate_angle(neck_point, hip_center)

    # print("바디바디222")
    
    # 각도 차이를 계산
    angle_diff = abs(head_neck_angle - neck_hip_angle)

    print(f"계산된 각도 차이: {angle_diff}")
    
    # 조건을 만족하면 True 반환
    if angle_diff < 165 and angle_diff > 145:  # 이 값은 조정 가능합니다.
        return True
    else:
        return False

In [12]:
def analyze_face_pose(keypoints):
    # 예제에서는 각 키포인트의 인덱스를 가정합니다.
    nose = keypoints[0]  # 코
    left_ear = keypoints[3]  # 왼쪽 귀
    right_ear = keypoints[4]  # 오른쪽 귀
    neck = keypoints[1]  # 목

    # 수평 변화 감지: 귀와 귀를 잇는 선의 각도
    horizontal_angle = calculate_angle(left_ear[:2], right_ear[:2])
    print(f"수평 변화 각도: {horizontal_angle}도")

    # 높이 변화 감지: 코와 목 사이의 거리
    vertical_distance = distance(nose[:2], neck[:2])
    print(f"높이 변화 거리: {vertical_distance}")

    # 수직 변화 감지: 코와 목을 잇는 선의 각도
    vertical_angle = calculate_angle(nose[:2], neck[:2])
    print(f"수직 변화 각도: {vertical_angle}도")

    # 좌우 변화 감지: 코와 양쪽 귀의 중점을 잇는 선의 각도
    ears_midpoint = ((left_ear[0] + right_ear[0]) / 2, (left_ear[1] + right_ear[1]) / 2)
    rotation_angle = calculate_angle(nose[:2], ears_midpoint)
    print(f"좌우 변화 각도: {rotation_angle}도")

In [13]:
def distance(p1, p2):
    """두 점 p1, p2 사이의 거리를 계산합니다."""
    return np.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)

In [14]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [15]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 316,
  'shape': array([ 1,  1, 17,  3]),
  'shape_signature': array([ 1,  1, 17,  3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [16]:
shaped = np.squeeze(np.multiply(interpreter.get_tensor(interpreter.get_output_details()[0]['index']), [480,640,1]))

In [17]:
for edge, color in EDGES.items():
    p1, p2 = edge
    y1, x1, c1 = shaped[p1]
    y2, x2, c2 = shaped[p2]
    print((int(x2), int(y2)))

(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)


In [18]:
for kp in shaped:
    ky, kx, kp_conf = kp
    print(int(ky), int(kx), kp_conf)

0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0
0 0 0.0


# 2. Make Detections

In [19]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    
    # input_image = tf.cast(img, dtype=tf.float32)
    # 모델이 요구하는 입력 타입에 맞게 타입 변환
    input_image = tf.cast(img, dtype=tf.uint8)  # dtype을 tf.float32에서 tf.uint8로 변경
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

    right_eye = keypoints_with_scores[0][0][2]
    left_elbow = keypoints_with_scores[0][0][7]

    # 키포인트 추출 및 조정
    keypoints = np.squeeze(np.multiply(keypoints_with_scores, [frame.shape[0], frame.shape[1], 1]))
    keypoints = keypoints[:, :2]  # x, y 좌표만 사용
    
    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    # 기존 렌더링 코드 아래에 세로선 그리기 함수 호출 추가
    draw_midline(frame, keypoints_with_scores, 0.4)

    # 얼굴 중심축 그리기 함수 호출
    draw_face_axis(frame, keypoints_with_scores, 0.4)

    # 얼굴 포즈 분석 함수 호출
    analyze_face_pose(keypoints)

    #   # 어드레스 자세 판별
    # if is_address_pose(keypoints_with_scores, 0.4):
    #     print("어드레스 자세 감지됨!")
    #     time.sleep(2)
    #     # 어드레스 자세가 감지되면 콘솔에 메시지 출력
    #     # 어드레스 자세 감지 시 수행할 추가 작업을 여기에 구현할 수 있습니다.
    # else:
    #     print("어드레스 자세 아님.")  # 어드레스 자세가 아니면 콘솔에 메시지 출력
    #     time.sleep(2)

    if is_address_pose(keypoints_with_scores, 0.4) and is_body_straight(keypoints_with_scores, 0.4):
        print("어드레스 자세 감지됨!")  # 어드레스 자세와 몸통이 똑바로 서 있는 상태 감지
        time.sleep(1)
    else:
        print("어드레스 자세 아님.")
        time.sleep(1)
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

수평 변화 각도: -86.18524325730559도
높이 변화 거리: 17.110164606146864
수직 변화 각도: 124.09125805149334도
좌우 변화 각도: 91.44635089698279도
어드레스 자세 아님.
수평 변화 각도: -85.367509443583도
높이 변화 거리: 14.671972926218723
수직 변화 각도: 127.25120001241672도
좌우 변화 각도: 31.255596127881745도
어드레스 자세 아님.
수평 변화 각도: -85.80949536424815도
높이 변화 거리: 14.287004967262595
수직 변화 각도: 132.02891598459522도
좌우 변화 각도: 14.63766693468851도
어드레스 자세 아님.
수평 변화 각도: -86.2691118766578도
높이 변화 거리: 15.020879001159576
수직 변화 각도: 133.43227813442974도
좌우 변화 각도: 54.68561403769461도
어드레스 자세 아님.
수평 변화 각도: -88.14775385297327도
높이 변화 거리: 17.74675423091312
수직 변화 각도: 129.57227882158415도
좌우 변화 각도: 174.81981153676952도
어드레스 자세 아님.
수평 변화 각도: -90.9705097917453도
높이 변화 거리: 21.443898873692113
수직 변화 각도: 132.1986514168518도
좌우 변화 각도: 172.91464479211396도
어드레스 자세 아님.
수평 변화 각도: -88.19783576239935도
높이 변화 거리: 18.073788385173824
수직 변화 각도: 128.05278445235126도
좌우 변화 각도: 160.79844376526677도
어드레스 자세 아님.
수평 변화 각도: -88.67554113212152도
높이 변화 거리: 20.383235032086507
수직 변화 각도: 120.15833156517095도
좌우 

### 머리-목 축과 목-골반 축의 각 계산하여 어드레스 자세 감지



아이언 어드레스 -> 어드레스 자세 아님

드라이버 어드레스  -> 어드레스 자세 감지됨!

(angle_diff 범위 수정하여 특정 자세 감지하도록 설정)

print()문이 어깨 + 골반 관절 감지 안했을때 안나오는 이유??